# Machine Learning Foundation

## Section 2, Part e: Regularization LAB 

## Learning objectives

By the end of this lesson, you will be able to:

* Implement data standardization
* Implement variants of regularized regression
* Combine data standardization with the train-test split procedure
* Implement regularization to prevent overfitting in regression problems

In [1]:
import numpy as np
import pandas as pd
from helper import boston_dataframe

np.set_printoptions(precision=3, suppress=True)

### Loading in Boston Data

**Note:** See `helper.py` file to see how boston data is read in from SciKit Learn.

In [2]:
boston = boston_dataframe(description=True)
boston_data = boston[0]
boston_description = boston[1]

In [3]:
boston_data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


## Data standardization

**Standardizing** data refers to transforming each variable so that it more closely follows a **standard** normal distribution, with mean 0 and standard deviation 1.

The [`StandardScaler`](http://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) object in SciKit Learn can do this.

**Generate X and y**:

In [4]:
y_col = "MEDV"

X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

**Import, fit, and transform using `StandardScaler`**

In [5]:
from sklearn.preprocessing import StandardScaler

s = StandardScaler()
X_ss = s.fit_transform(X)

### Exercise: 

Confirm standard scaling

Hint:

In [6]:
a = np.array([[1, 2, 3], 
              [4, 5, 6]]) 
print(a) # 2 rows, 3 columns

[[1 2 3]
 [4 5 6]]


In [32]:
a.mean(axis=0) # mean along the *columns*

array([2.5, 3.5, 4.5])

In [33]:
a.mean(axis=1) # mean along the *rows*

array([2., 5.])

In [34]:
### BEGIN SOLUTION
X2 = np.array(X)
man_transform = (X2-X2.mean(axis=0))/X2.std(axis=0)
np.allclose(man_transform, X_ss)
### END SOLUTION

True

### Coefficients with and without scaling

In [35]:
from sklearn.linear_model import LinearRegression

In [36]:
lr = LinearRegression()

y_col = "MEDV"

X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

In [37]:
lr.fit(X, y)
print(lr.coef_) # min = -18

[ -0.108   0.046   0.021   2.687 -17.767   3.81    0.001  -1.476   0.306
  -0.012  -0.953   0.009  -0.525]


#### Discussion (together): 

The coefficients are on widely different scales. Is this "bad"?

In [38]:
from sklearn.preprocessing import StandardScaler

In [39]:
s = StandardScaler()
X_ss = s.fit_transform(X)

In [15]:
lr2 = LinearRegression()
lr2.fit(X_ss, y)
print(lr2.coef_) # coefficients now "on the same scale"

[-0.928  1.082  0.141  0.682 -2.057  2.674  0.019 -3.104  2.662 -2.077
 -2.061  0.849 -3.744]


### Exercise: 

Based on these results, what is the most "impactful" feature (this is intended to be slightly ambiguous)? "In what direction" does it affect "y"?

**Hint:** Recall from last week that we can "zip up" the names of the features of a DataFrame `df` with a model `model` fitted on that DataFrame using:

```python
dict(zip(df.columns.values, model.coef_))
```

In [41]:
### BEGIN SOLUTION
pd.DataFrame(zip(X.columns, lr2.coef_)).sort_values(by=1)
### END SOLUTION

,0,1
12,LSTAT,-3.743627
7,DIS,-3.104044
9,TAX,-2.076782
10,PTRATIO,-2.060607
4,NOX,-2.056718
0,CRIM,-0.928146
6,AGE,0.019466
2,INDUS,0.140900
3,CHAS,0.681740
11,B,0.849268


Looking just at the strength of the standardized coefficients LSTAT, DIS, RM and RAD are all the 'most impactful'. Sklearn does not have built in statistical signifigance of each of these variables which would aid in making this claim stronger/weaker

### Lasso with and without scaling

We discussed Lasso in lecture. 

Let's review together:

1. What is different about Lasso vs. regular Linear Regression?
1. Is standardization more or less important with Lasso vs. Linear Regression? Why?

In [18]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures

#### Create polynomial features

[`PolynomialFeatures`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

In [48]:
pf = PolynomialFeatures(degree=2, include_bias=False,)
X_pf = pf.fit_transform(X)


**Note:** We use `include_bias=False` since `Lasso` includes a bias by default.

In [49]:
X_pf_ss = s.fit_transform(X_pf)

### Lasso

[`Lasso` documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)

In [50]:
las = Lasso()
las.fit(X_pf_ss, y)
las.coef_ 

array([-0.   ,  0.   , -0.   ,  0.   , -0.   ,  0.   , -0.   , -0.   ,
       -0.   , -0.   , -0.991,  0.   , -0.   , -0.   ,  0.   , -0.   ,
        0.068, -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   ,
       -0.   , -0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.   ,  0.   ,
       -0.   , -0.   , -0.   , -0.05 , -0.   , -0.   , -0.   , -0.   ,
       -0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   , -0.   , -0.   , -0.   , -0.   , -0.   ,
       -0.   , -0.   ,  0.   , -0.   ,  3.3  , -0.   , -0.   , -0.   ,
       -0.   , -0.   ,  0.42 , -3.498, -0.   , -0.   , -0.   , -0.   ,
       -0.   ,  0.   , -0.   , -0.   , -0.   , -0.146, -0.   , -0.   ,
       -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   ,
       -0.   , -0.   , -0.   ,  0.   , -0.   ,  0.   , -0.   , -0.   ])

### Exercise

Compare 

* Sum of magnitudes of the coefficients
* Number of coefficients that are zero

for Lasso with alpha 0.1 vs. 1.

Before doing the exercise, answer the following questions in one sentence each:

* Which do you expect to have greater magnitude?
* Which do you expect to have more zeros?

In [51]:
### BEGIN SOLUTION
las01 = Lasso(alpha = 0.1)
las01.fit(X_pf_ss, y)
print('sum of coefficients:', abs(las01.coef_).sum() )
print('number of coefficients not equal to 0:', (las01.coef_!=0).sum())

sum of coefficients: 26.138682362877965
number of coefficients not equal to 0: 23


In [52]:
las1 = Lasso(alpha = 1)
las1.fit(X_pf_ss, y)
print('sum of coefficients:',abs(las1.coef_).sum() )
print('number of coefficients not equal to 0:',(las1.coef_!=0).sum())
### END SOLUTION

sum of coefficients: 8.47240504455307
number of coefficients not equal to 0: 7


With more regularization (higher alpha) we will expect the penalty for higher weights to be greater and thus the coefficients to be pushed down. Thus a higher alpha means lower magnitude with more coefficients pushed down to 0.

### Exercise: $R^2$

Calculate the $R^2$ of each model without train/test split.

Recall that we import $R^2$ using:

```python
from sklearn.metrics import r2_score
```

In [30]:
### BEGIN SOLUTION
from sklearn.metrics import r2_score
r2_score(y,las1.predict(X_pf_ss))
### END SOLUTION

0.7207000417838496

In [31]:
### BEGIN SOLUTION
from sklearn.metrics import r2_score
r2_score(y,las01.predict(X_pf_ss))
### END SOLUTION

0.8366045990855127

#### Discuss:

Will regularization ever increase model performance if we evaluate on the same dataset that we trained on?

## With train/test split

#### Discuss

Are there any issues with what we've done so far?

**Hint:** Think about the way we have done feature scaling.

Discuss in groups of two or three.

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X_pf, y, test_size=0.3, 
                                                    random_state=72018)

In [55]:
X_train_s = s.fit_transform(X_train)
las.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred = las.predict(X_test_s)
r2_score(y_pred, y_test)

0.33177406838134416

In [57]:
X_train_s = s.fit_transform(X_train)
las1.fit(X_train_s, y_train)
X_test_s = s.fit_transform(X_test)
y_pred = las1.predict(X_test_s)
r2_score(y_pred, y_test)

0.34947124693080767

In [58]:
X_train_s = s.fit_transform(X_train)
las01.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred = las01.predict(X_test_s)
r2_score(y_pred, y_test)

0.7102444090776476

### Exercise


#### Part 1:

Do the same thing with Lasso of:

* `alpha` of 0.001
* Increase `max_iter` to 100000 to ensure convergence. 

Calculate the $R^2$ of the model.

Feel free to copy-paste code from above, but write a one sentence comment above each line of code explaining why you're doing what you're doing.

#### Part 2:

Do the same procedure as before, but with Linear Regression.

Calculate the $R^2$ of this model.

#### Part 3: 

Compare the sums of the absolute values of the coefficients for both models, as well as the number of coefficients that are zero. Based on these measures, which model is a "simpler" description of the relationship between the features and the target?

In [59]:
### BEGIN SOLUTION

# Part 1

# Decreasing regularization and ensuring convergence
las001 = Lasso(alpha = 0.001, max_iter=100000)

# Transforming training set to get standardized units
X_train_s = s.fit_transform(X_train)

# Fitting model to training set
las001.fit(X_train_s, y_train)

# Transforming test set using the parameters defined from training set
X_test_s = s.transform(X_test)

# Finding prediction on test set
y_pred = las001.predict(X_test_s)

# Calculating r2 score
print("r2 score for alpha = 0.001:", r2_score(y_pred, y_test))


# Part 2

# Using vanilla Linear Regression
lr = LinearRegression()

# Fitting model to training set
lr.fit(X_train_s, y_train)

# predicting on test set
y_pred_lr = lr.predict(X_test_s)

# Calculating r2 score
print("r2 score for Linear Regression:", r2_score(y_pred_lr, y_test))


# Part 3
print('Magnitude of Lasso coefficients:', abs(las001.coef_).sum())
print('Number of coeffients not equal to 0 for Lasso:', (las001.coef_!=0).sum())

print('Magnitude of Linear Regression coefficients:', abs(lr.coef_).sum())
print('Number of coeffients not equal to 0 for Linear Regression:', (lr.coef_!=0).sum())
### END SOLUTION

r2 score for alpha = 0.001: 0.8686454101886707
r2 score for Linear Regression: 0.8555202098064174
Magnitude of Lasso coefficients: 436.26164263065647
Number of coeffients not equal to 0 for Lasso: 89
Magnitude of Linear Regression coefficients: 1185.285825446829
Number of coeffients not equal to 0 for Linear Regression: 104


## L1 vs. L2 Regularization

As mentioned in the deck: `Lasso` and `Ridge` regression have the same syntax in SciKit Learn.

Now we're going to compare the results from Ridge vs. Lasso regression:

[`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)

In [60]:
from sklearn.linear_model import Ridge

### Exercise

Following the Ridge documentation from above:

1. Define a Ridge object `r` with the same `alpha` as `las001`.
2. Fit that object on `X` and `y` and print out the resulting coefficients.

In [61]:
### BEGIN SOLUTION
# Decreasing regularization and ensuring convergence
r = Ridge(alpha = 0.001)
X_train_s = s.fit_transform(X_train)
r.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_r = r.predict(X_test_s)

# Calculating r2 score
r.coef_
### END SOLUTION

array([  8.727,  10.354, -24.683,   5.299,  -3.13 ,  14.781,  22.301,
       -23.483,  27.356,  -1.512,  16.949,  22.531,  11.444,   1.055,
         0.437,  14.259,   1.83 ,  -8.737,   4.726,  -3.748,  -3.557,
       -16.521,  -6.918,   6.064,  -1.363,   4.604,  -1.303,  -0.057,
        -0.301, -12.828,   2.016,   1.084,  -0.655,  -1.092,   4.458,
        -4.247,   4.156,  -1.282,   8.71 ,  -0.285,  -8.595,  11.595,
         6.544,   1.271,   1.827,   1.007,   1.554,   5.173,  -4.685,
         5.299,  -3.199,  -8.715,   0.983,   1.119,   0.284,  -1.683,
        -2.884,   2.985,  -0.756,  12.297,   0.931,  -7.594,  18.292,
       -22.124,  35.725, -21.894,  -7.789,   1.774,   4.101, -12.263,
        -3.741,  -5.525, -16.111,  -5.706,  -4.666,  -9.884,   0.989,
        -0.092,  17.145, -14.214,  -2.955,  -2.785,  -5.404,  11.825,
         0.085,  -4.352,  -5.616,  -2.843,   0.826, -29.129,  49.636,
       -21.571,  -1.263,  -8.811, -15.822,  11.975,  -1.023,   2.911,
        -0.656,  -4.

In [62]:
las001 # same alpha as Ridge above

Lasso(alpha=0.001, max_iter=100000)

In [63]:
las001.coef_

array([  0.   ,   0.   , -16.945,   2.562,   0.   ,  13.392,   9.907,
       -19.96 ,   9.039,   0.   ,   6.138,  17.628,  11.446,   1.21 ,
         0.226,  11.578,   2.197,  -7.397,   4.117,  -1.589,  -2.157,
        -9.837,   0.   ,   0.   ,  -1.089,   3.793,   0.389,   0.2  ,
        -0.297,  -3.352,   0.396,   0.719,  -0.756,  -0.753,   2.388,
        -0.845,   2.524,  -0.973,   3.882,  -0.983,   6.609,   6.138,
         4.201,   0.858,   1.96 ,   2.661,  -4.148,   2.937,  -4.556,
         2.112,  -1.983,  -7.439,   1.607,   1.668,  -1.262,  -0.   ,
        -0.   ,   2.622,  -0.861,   3.353,  -0.   ,  -2.451,   9.171,
        -6.917,   0.   ,  -5.618,  -5.716,   0.521,   3.354,  -7.528,
        -0.   ,  -6.646, -10.212,  -6.346,  -2.95 ,  -9.559,   0.327,
         0.56 ,  11.006,  -6.666,  -1.158,  -2.25 ,  -3.53 ,   9.035,
        -0.   ,  -2.316,  -2.   ,  -1.124,   0.804, -20.861,  27.166,
         0.   ,  -1.51 ,  -6.745,  -5.098,  15.455,   0.   ,   0.   ,
         0.629,  -3.

In [64]:
print(np.sum(np.abs(r.coef_)))
print(np.sum(np.abs(las001.coef_)))

print(np.sum(r.coef_ != 0))
print(np.sum(las001.coef_ != 0))

795.6521694490916
436.26164263065647
104
89


**Conclusion:** Ridge does not make any coefficients 0. In addition, on this particular dataset, Lasso provides stronger overall regularization than Ridge for this value of `alpha` (not necessarily true in general).

In [65]:
y_pred = r.predict(X_pf_ss)
print(r2_score(y, y_pred))

y_pred = las001.predict(X_pf_ss)
print(r2_score(y, y_pred))

0.907527834057458
0.9102933722688233


**Conclusion**: Ignoring issues of overfitting, Ridge does slightly better than Lasso when `alpha` is set to 0.001 for each (not necessarily true in general).

# Example: Does it matter when you scale?

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X_ss, y, test_size=0.3, 
                                                    random_state=72018)

In [67]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_pred, y_test)

0.5832942094971805

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=72018)

In [69]:
s = StandardScaler()
lr_s = LinearRegression()
X_train_s = s.fit_transform(X_train)
lr_s.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_s = lr_s.predict(X_test_s)
r2_score(y_pred_s, y_test)

0.5832942094971799

**Conclusion:** It doesn't matter whether you scale before or afterwards, in terms of the raw predictions, for Linear Regression. However, it matters for other algorithms. Plus, as we'll see later, we can make scaling part of a `Pipeline`.

# What about Lasso ?

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X_ss, y, test_size=0.3, 
                                                    random_state=72018)

In [80]:
lasso = Lasso()
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)
r2_score(y_pred, y_test)

0.2067158380521159

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=72018)

In [82]:
s = StandardScaler()
lasso = Lasso()
X_train_s = s.fit_transform(X_train)
lasso.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_s = lasso.predict(X_test_s)
r2_score(y_pred_s, y_test)

0.20880736283132462

Yes, there is difference when we do Scalar Standarization after splitting the data, the accuracy gets higher 

---
### Machine Learning Foundation (C) 2020 IBM Corporation